# ENGSCI 700 Geothermal Reservoir Optimisation

This workbook is for extracting Contact well data and recreating the plots.

(Unix) launch with `cd src` >`jupyter notebook`

File structure: 
```
(root)
├── src
│    └── Python Test.ipynb
└── wairakei_data
     └── Liquid wells (version 1).xlsx
```

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
import pickle
from datetime import datetime, timedelta
%matplotlib inline

base_year = '2000'    # numeric dates calculated from Jan-01

def datetime_to_numeric(my_datetime):
    # returns days since base_year-01-01.
    try:
        date_numeric = (my_datetime - datetime(int(base_year),1,1)) / timedelta(days=1)
    except:
        date_numeric = (my_datetime - np.datetime64(base_year)) / np.timedelta64(1, 'D')
#         date_numeric = [(x - np.datetime64(base_year)) / np.timedelta64(1, 'D') for x in list(my_datetime)]
    return date_numeric

# Check if Excel file is already in memory (loading is slow)
try:    xl
except: xl = pd.ExcelFile('../wairakei_data/Liquid wells (version 1).xlsx')
print([x for x in xl.sheet_names if len(x) < 6], '...')

## Prepare data

In [ ]:
import re

# sheets to load data from
# wells = ['wk255', 'wk256', 'w258', 'w259']
sheets = ['wk255', 'w258']
# wells = ['wk255', 'wk256', 'wk247', 'w267', 'w268',
#          'w269', 'WK270', 'WK271', 'WK272']
        # missing: wk251, wk250, wk252, wk238, wk234, wk240, wk241, wk267, wk268

dfs = []
for sheet in sheets:
    df = xl.parse(sheet)                                       # select well data
    df['well'] = sheet                                            # label data with well name
    dfs.append(df)
df = pd.concat(dfs)

# df = df.loc[:, ~df.columns.str.contains('^Unnamed|SUMMARY|slope|intercept')]     # remove extra columns
df = df[['date', 'whp', 'mf', 'h', 'well']]                      # instead, only keep certain columns
df['well'] = df['well'].str.lower()                              # remove 'WK' inconsistencies
df['well'] = df['well'].str.replace("^[^\d]*", "wk")
df['mf'] = pd.to_numeric(df['mf'], errors='coerce')              # remove 'dummy' entries
df = df.dropna(subset=['date', 'whp', 'mf'])                     # remove NA

df['date_numeric'] = datetime_to_numeric(df['date']) #  yrs since base_year
df = df.reset_index()
# save data to import into R if needed
df.to_csv('../wairakei_data/data.csv', index=False)
wells = df['well'].unique()
df.head()

## Create exploratory plots

In [ ]:
cmap = plt.get_cmap('viridis')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[14,4])
marker = itertools.cycle(['o', ',', '+', 'x', '*', '.'])
ax1.set_xlabel('date')
ax1.set_ylabel('whp')
ax2.set_xlabel('whp')
ax2.set_ylabel('mf')

# right plot
for well in wells:
    mkr = next(marker)
    ax1.scatter('date_numeric', 'whp', data=df.loc[df['well']==well], marker=mkr, label=well)
    ax2.scatter('whp', 'mf', c='date_numeric', data=df.loc[df['well']==well], marker=mkr, label=well)
ax1.legend()
ax2.legend()

plt.show()

## Set up regression data and create prediction frame for plotting

In [ ]:
date_pred = np.arange(df['date'].min(), df['date'].max(), np.timedelta64(365*2, 'D').astype(datetime))
date_numeric_pred = datetime_to_numeric(date_pred)
whp_pred = np.linspace(0, 16, 1000)
well_pred = wells
pred = pd.DataFrame(list(itertools.product(date_numeric_pred, whp_pred, well_pred)),
                    columns=['date_numeric', 'whp', 'well'])
pred.head()

## Perform regression and prediction

In [ ]:
from statsmodels.formula.api import ols

# Not conditioned on date
model1 = ols("mf ~ well * whp", data=df)
results1 = model1.fit()
pred['mf1'] = results1.predict(pred)

# Linear fit dependent on date
model2 = ols("mf ~ well * (whp + date_numeric)", data=df)
results2 = model2.fit()
pred['mf2'] = results2.predict(pred)

# Elliptic fit dependent on date
model3 = ols("np.power(mf,2) ~ well * (np.power(whp,2) + date_numeric)", data=df)
results3 = model3.fit()
# pred = pd.DataFrame({"well": well, "whp": whp, "date_numeric":date})
pred['mf3^2'] = results3.predict(pred)
pred.loc[pred['mf3^2'] < 0, 'mf3^2'] = np.nan    # remove invalid results
pred['mf3'] = np.sqrt(pred['mf3^2'])

# save regression
def save_object(obj, filename, verbose=0):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    if verbose:
        print("Saved %s to '%s'" %(obj, filename))
        
save_object(results3, 'regression.pkl', 1)

## Create plots

In [ ]:
# ===============================================================
# Set up axes
# ===============================================================

from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase

# colors
indices = np.linspace(0, cmap.N, len(df))
my_colors = [cmap(int(i)) for i in indices]

# subplots
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=[14,4], gridspec_kw={'width_ratios': [9,9,9,1]})
ax1.get_shared_y_axes().join(ax1, ax2, ax3)
ax1.set_ylim([0, 1000])
ax1.set_xlim(0, 16)
ax1.set_ylabel('Mass flow')
ax1.set_xlabel("Well head pressure")
ax1.set_title('$mf \sim whp$')
ax2.set_title('$mf \sim whp + date$')
ax3.set_title('$mf^2 \sim whp^2 + date$')

# create date colorbar
indices = np.linspace(0, cmap.N, len(date_pred))
my_colors = [cmap(int(i)) for i in indices]
norm = Normalize(np.min(df['date']).year, np.max(df['date']).year)
cb = ColorbarBase(ax4, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label('Year')

linestyles = itertools.cycle(('-', '--', '-.', ':'))

# ===============================================================
# Plot data
# ===============================================================

# plot raw data points
marker.__init__()
for well in wells:
    mkr = next(marker)
    for ax in [ax1, ax2, ax3]:
        ax.scatter('whp', 'mf', c='date_numeric', data=df.loc[df['well']==well], marker=mkr, label=well)
    
# plot fitted curves
for well in wells:
    lty = next(linestyles)
    # model 1
    # 'data' argument filters the data to just the data from one well, using a single date
    ax1.plot('whp', 'mf1', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==np.min(pred['date_numeric']))])

    # model 2 & 3
    for i, date in enumerate(date_numeric_pred):
        # 'data' argument similar, for a specific prediction date in the loop
        ax2.plot('whp', 'mf2', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==date)], c=my_colors[i])
        ax3.plot('whp', 'mf3', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==date)], c=my_colors[i])

# show model selection criteria
for ax, result in zip([ax1, ax2, ax3], [results1, results2, results3]):
    ax.legend(['Adj $R^2$: %.2f' % result.rsquared_adj,
               'AIC: %.2f' % result.aic], 
              handlelength=0, handletextpad=0, loc=1).legendHandles[0].set_visible(False)

## Set up PyJAGS

In [ ]:
import pyjags
import warnings
from sklearn.model_selection import ParameterGrid


class MyModel(pyjags.Model):
    """
    Create my own model child class that doesn't ValueError on unused variables
    """
    def _init_compile(self, data, generate_data):
        if data is None:
            data = {}
        data = pyjags.model.dict_to_jags(data)
        unused = set(data.keys()) - set(self.variables)
        if unused:
            warnings.warn('Unused data for variables: {}'.format(','.join(unused)))
        self.console.compile(data, self.chains, generate_data)
        

def make_prediction_data(whp_pred):
    data_df = df.copy()
    
    date_numeric_pred = [datetime_to_numeric(datetime.now())]
    try: iter(whp_pred)
    except TypeError: whp_pred = [whp_pred]
    else: whp_pred = list(whp_pred)
    mf_pred = [np.nan]
    well_pred = df['well'].unique()
    X = list(ParameterGrid({'date_numeric': date_numeric_pred,
                            'whp': whp_pred,
                            'mf': mf_pred,
                            'well': well_pred}))
    empty_rows = {key: [x[key] for x in X] for key in X[0].keys()}
    
    data_df = data_df.append(pd.DataFrame(empty_rows)).reset_index()
    data = data_df.to_dict('list')
    data['well_id'] = data_df['well'].factorize()[0] + 1
    data['m'] = len(data['well'].unique())
    data['n_init'] = len(df)
    data['n'] = len(data_df)
    for key in ['index', 'h', 'date', 'well']:
        # rm unused variables
        del data[key]
    for key in data.keys():
        # prepare np.nans for prediction
        data[key] = np.ma.masked_invalid(data[key])
    return data

code = '''
model {
  # find means
  mean_whp <- mean(whp)
  mean_date_numeric <- mean(date_numeric)
  
  # transform to centered covariates
  c_whp <- whp - mean_whp
  c_date_numeric <- date_numeric - mean_date_numeric
  
  # fit individual regressions to wells
  for (i in 1:n) {
    mu[i] <- sqrt(abs(mu2[i])) + beta_date_numeric[well_id[i]] * c_date_numeric[i]
    mu2[i] <- Intercept[well_id[i]] + beta_whp[well_id[i]] * pow(c_whp[i], 2)
    #mu[i] <- Intercept[well_id[i]] + beta_whp[well_id[i]] * c_whp[i] + beta_date_numeric[well_id[i]] * c_date_numeric[i]
    mf[i] ~ dnorm(mu[i], tau)
  }

  I ~ dnorm(0, 1e-8)
  b_w ~ dnorm(0, 1e-8)
  b_d_n ~ dnorm(0, 1e-8)
  
  for (j in 1:m) {
    Intercept[j] ~ dnorm(0, 1e-8)
    beta_whp[j] ~ dnorm(0, 1e-8)
    beta_date_numeric[j] ~ dnorm(0, 1e-6)
  }

  tau ~ dgamma(1e-6, 1e-6)

  # make predictions
  wk255_pred <- mu[n-1]
  wk256_pred <- mu[n]
}
'''

def run_model(whp, extra_vars=[]):
    # generate data with nan entries for prediction
    if whp is not None:
        data = make_prediction_data(whp)
    else:
        raise Exception("Error: Don't use band data yet")
    
    n_chains = 1
    burn_in = 100
    n_samples = 500
    model = MyModel(code, data=data, chains=n_chains)
    model.sample(burn_in, vars=[])
    samples = model.sample(n_samples, vars=['mu'])
    samples = {key: values.squeeze() for key, values in samples.items()}
    
    # create sample frame with info from data
    return_data = {key: [] for key in data.keys() if np.size(data[key]) == data['n']}
    for key in return_data.keys():
        return_data[key] = np.array(data[key][data['n_init']:data['n']])
    return_data['mf'] = samples['mu'][data['n_init']:data['n']]
    
    # add back metadata
    return_data['n'] = data['n'] - data['n_init']
    return_data['m'] = data['m']
    return_data['n_samples'] = n_samples
    
    return return_data

# def summary(samples, varname, p=95):
#     values = samples[varname]
#     ci = np.percentile(values, [100-p, p])
#     print('{:<6} mean = {:>5.1f}, {}% credible interval [{:>4.1f} {:>4.1f}]'.format(
#       varname, np.mean(values), p, *ci))

In [ ]:
from ipywidgets import FloatSlider

def plot_everything(whp=None):
    fig, (ax, bar) = plt.subplots(1, 2, figsize=[14,4], gridspec_kw={'width_ratios': [9,1]})
    ax.set_ylim([0, 1000])
    ax.set_title('$mf \sim whp + date$')
    ax.set_ylabel('Mass flow')
    ax.set_xlabel("Well head pressure")
    ax.set_xlim(0, 16)

    # create date colorbar
    indices = np.linspace(0, cmap.N, len(date_pred))
    my_colors = [cmap(int(i)) for i in indices]
    norm = Normalize(np.min(df['date']).year, np.max(df['date']).year)
    cb = ColorbarBase(bar, cmap=cmap, norm=norm, orientation='vertical')
    cb.set_label('Year')

    linestyles = itertools.cycle(('-', '--', '-.', ':'))

    # ===============================================================
    # Plot data
    # ===============================================================
    # plot raw data points
    marker.__init__()
    for well in wells:
        mkr = next(marker)
        ax.scatter('whp', 'mf', c='date_numeric', data=df.loc[df['well']==well], marker=mkr, label=well)

    # plot linreg lines
    for well in wells:
        lty = next(linestyles)
        for i, date in enumerate(date_numeric_pred):
            ax.plot('whp', 'mf3', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==date)],
                    c=my_colors[i], alpha=0.5)

    samples = run_model(whp)
    if whp is not None:
        # plot predicted values
        for i in range(samples['n']):
            ax.violinplot(samples['mf'][i], positions=[samples['whp'][i]], showmeans=True, showextrema=False)
    
# widgets.interact(plot_everything, whp=(0., 16.), continuous_update=False);
from ipywidgets import interact, FloatSlider
interact(plot_everything, whp=FloatSlider(min=0, max=16, value=12, continuous_update=False));

## Put into confidence band form (not actually used for anything)

In [ ]:
fig, (ax, bar) = plt.subplots(1, 2, figsize=[14,4], gridspec_kw={'width_ratios': [9,1]})
ax.set_ylim([0, 1000])
ax.set_title('$mf \sim whp + date$')
ax.set_ylabel('Mass flow')
ax.set_xlabel("Well head pressure")
ax.set_xlim(0, 16)

# create date colorbar
indices = np.linspace(0, cmap.N, len(date_pred))
my_colors = [cmap(int(i)) for i in indices]
norm = Normalize(np.min(df['date']).year, np.max(df['date']).year)
cb = ColorbarBase(bar, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label('Year')

linestyles = itertools.cycle(('-', '--', '-.', ':'))

# ===============================================================
# Plot data
# ===============================================================

# plot raw data points
marker.__init__()
for well in wells:
    mkr = next(marker)
    ax.scatter('whp', 'mf', c='date_numeric', data=df.loc[df['well']==well], marker=mkr, label=well)

# plot linreg lines
for well in wells:
    lty = next(linestyles)
    for i, date in enumerate(date_numeric_pred):
        ax.plot('whp', 'mf2', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==date)],
                c=my_colors[i], alpha=0.5)

samples = run_model(list(range(16)))
for well_id in range(1, samples['m']+1):
    idx = np.where(samples['well_id'] == well_id)[0]
    # create temporary dict for plotting
    regplot_data = {'whp': [], 'mf': []}
    for i in idx:
        regplot_data['whp'] += [samples['whp'][i]] * samples['n_samples']
        regplot_data['mf'] += list(samples['mf'][i])
        sns.regplot('whp', 'mf', pd.DataFrame(regplot_data), scatter=False, ax=ax)

## Introduce Flash Plants

In [ ]:
try:    fpxl
except: fpxl = pd.ExcelFile('../wairakei_data/Data for AU.xlsx')
    
fpdf = pd.read_excel(fpxl, 'calculation', header=1, usecols="D:T")
fpdf = fpdf.rename(columns={"FP15": "well", "Unnamed: 1": "fp"})
fpdf = fpdf[pd.to_numeric(fpdf['whp'], errors='coerce').notnull()]
for col in ['well', 'fp']:
    fpdf[col] = fpdf[col].str.lower()
fpdf[fpdf.columns] = fpdf[fpdf.columns].apply(pd.to_numeric, errors='ignore')
fpdf.head()

What are the operating conditions for each well?

In [ ]:
# find details of the last known operating conditions
last_idx = df.groupby('well')['date_numeric'].idxmax()
df.iloc[last_idx]

In [ ]:
# set mapping of wells to FPs
well_fp_map = {'wk255': 'fp15',
               'wk256': 'fp15'}

map_details = fpdf.loc[(fpdf['well'].isin(well_fp_map.keys())) & (fpdf['fp'].isin(well_fp_map.values()))]
map_details

In [ ]:
code_fp = '''
model {
  # find means
  mean_whp <- mean(whp)
  mean_date_numeric <- mean(date_numeric)
  
  # transform to centered covariates
  c_whp <- whp - mean_whp
  c_date_numeric <- date_numeric - mean_date_numeric
  
  # fit individual regressions to wells
  for (i in 1:n) {
    mu[i] <- Intercept[well_id[i]] + beta_whp[well_id[i]] * c_whp[i] + beta_date_numeric[well_id[i]] * c_date_numeric[i]
    mf[i] ~ dnorm(mu[i], tau)
  }
  
  for (j in 1:m) {
    Intercept[j] ~ dnorm(0, 1e-6)
    beta_whp[j] ~ dnorm(0, 1e-8)
    beta_date_numeric[j] ~ dnorm(0, 1e-6)
  }
  tau ~ dgamma(1e-6, 1e-6)
  
  # estimate mf at wells
  # requires n_init:n estimates to be in order of well id
  for (j in (n_init+1):n) {
    
  }
  
  # estimate steam at FPs
  for () {
    
  }
}
'''